Question
========

What's the best developer job like ? (depending on your own definition
of \"best\")

Dataset
=======

The dataset provided by [StackOverflow on
Kaggle](https://www.kaggle.com/stackoverflow/so-survey-2017/data) seems
to be a great start. It contains about fifty thousand answers from a
sample of the active StackOverflow population about a lot of questions,
namely 154. This means that we would have a tremendous insight into what
makes a programmer unique, but also can help us to answer a lot of
interesting questions.

Project
=======

Exploratory
-----------

Check the distributions of all useful features, outliers, quantiles.
Questions we could answer with the exploration:

-   Does salary equates to happiness/fulfilment in your job ?

-   For users not satisfied with their job, what should they change to
    be more satisfied (use closest correlated neighbor) ?

-   How much is Job Satisfaction linked to education ?

-   Are \"gif\" people more satisfied with their job compared to \"jif\"
    people ?
    
Metric
--------------
Derive metric to measure happiness/fulfillment

Pre-processing
--------------

Data cleaning, categorize values, check out their distribution,
selecting columns, removing bad values if needed.

Feature Extraction
------------------

PCA to check which are the features explaining the most variance.

Graph Analysis
--------------

The graph will be built the following way:

-   Users will be the nodes

-   Correlations (with a threshold) in-between users used as edges

Recommender System
------------------

The idea here would be to be able to recommend which of a set of users
best represents a set of given goals. To do so, we would simply check
which existing node is the closest to the artificial one that we create
for the chosen features a recruiter is looking for.


In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np 
import pandas as pd 
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()
from subprocess import check_output
pd.set_option('display.max_columns', None)

In [ ]:
stack = pd.read_csv("data/survey_results_public.csv")
kept_columns = ['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University', 'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'CompanySize', 'CompanyType', 'YearsProgram', 'YearsCodedJob', 'DeveloperType', 'WebDeveloperType', 'NonDeveloperType', 'CareerSatisfaction', 'JobSatisfaction', 'PronounceGIF', 'ProblemSolving', 'BuildingThings', 'LearningNewTech', 'BoringDetails', 'JobSecurity', 'DiversityImportant', 'FriendsDevelopers', 'WorkPayCare', 'ChallengeMyself', 'ImportantBenefits', 'ClickyKeys', 'Overpaid', 'TabsSpaces', 'EducationImportant', 'EducationTypes', 'SelfTaughtTypes', 'WorkStart', 'HaveWorkedLanguage', 'WantWorkLanguage', 'IDE', 'AuditoryEnvironment', 'Methodology', 'EquipmentSatisfiedMonitors', 'InfluenceTechStack', 'InfluenceCommunication', 'StackOverflowSatisfaction', 'StackOverflowFoundAnswer', 'StackOverflowCopiedCode', 'StackOverflowWhatDo', 'Gender', 'HighestEducationParents', 'Race', 'Salary']
stack = stack[kept_columns]
stack.set_index("Respondent", inplace=True)
stack.head()

In [ ]:
# We filter out the devs upon some criteria
def row_filter(row):
    # After checking salary values, we decided to remove the first 5%
    # quantile as they were mostly outliers (values inbetween 0 and 100)
    if row.isnull().Salary or row.Salary < stack.Salary.quantile(0.05):
        return False
    if row.Professional not in ["Student", 
                                "Professional developer", 
                                "Professional non-developer who sometimes writes code"]:
        return False
    if row.EmploymentStatus not in ['Employed part-time',
                                    'Employed full-time',
                                    'Independent contractor, freelancer, or self-employed']:
        return False
    if row.isnull().JobSatisfaction and row.isnull().CareerSatisfaction:
        return False
    return True
    
stack = stack[stack.apply(row_filter, axis=1)]

In [ ]:
metadata = pd.read_csv("data/survey_results_schema.csv")
metadata

# Exploratory Analysis

In this section we will explore different columns of our dataframe to have an idea of what the population we have looks like.

In [ ]:
import matplotlib.pyplot as plt

## Professional

In [ ]:
stack['Professional'].value_counts()[0:10].plot(kind='bar',figsize=(10,8))
plt.show()

TODO

## Country

In [ ]:
stack['Country'].value_counts()[0:10].plot(kind='bar',figsize=(10,8))
plt.show()

TODO

## Employment Status

In [ ]:
stack['EmploymentStatus'].value_counts()[0:10].plot(kind='bar',figsize=(10,8))
plt.show()

TODO

## Developer Type

In [ ]:
DevTypes = [lang for sublist in [str(langs).replace(" ", "").split(";") for langs in stack['DeveloperType']] for lang in sublist]
DevTypes = pd.Series(DevTypes)

In [ ]:
DevTypes.value_counts()[0:10].plot(kind='bar',figsize=(10,8))
plt.show()

TODO

## Career Statisfaction

In [ ]:
carrSat = stack['CareerSatisfaction']/stack['CareerSatisfaction'].max()

In [ ]:
carrSat.value_counts().sort_index().plot(kind='bar',figsize=(10,8))
plt.show()

## GIF vs JIF

In [ ]:
import matplotlib.pyplot as plt
temp = stack[["PronounceGIF", "Salary"]].dropna(how='any')
temp = temp.set_index("PronounceGIF")
gif = temp.loc['With a hard "g," like "gift"'].Salary.values
jif = temp.loc['With a soft "g," like "jiff"'].Salary.values
tr = pd.DataFrame()
tr['gif'] = pd.Series(gif)
filling = np.empty((6081))
filling[:] = np.nan
to_add = np.append(jif, filling)
tr['jif'] = pd.Series(to_add)
plot = sns.boxplot(data=tr, orient="v",)
plt.ylabel("Salary")
plt.title("Distribution of salary for the gif and jif populations")
plt.show()

print(tr.describe())